In [1]:
import nltk
import re
import os
import pandas as pd
import numpy as np
from pymystem3 import Mystem
from string import punctuation, digits

In [2]:
%%time

names = ['filename', 'title', 'year', 'author', 'years_of_life',
         'time_summary', 'time_book', 'name', 'username', 'tradition_country']
df = pd.read_csv('metatable.tsv', sep='\t', names=names)

CPU times: user 24.3 ms, sys: 4.36 ms, total: 28.7 ms
Wall time: 46.3 ms


In [3]:
%%time

split_tradition_country = df['tradition_country'].str.split('->')
df['tradition']= split_tradition_country.str.get(0).apply(lambda x: x.replace('\xad', ''))
df['country']=split_tradition_country.str.get(1)

CPU times: user 9.82 ms, sys: 0 ns, total: 9.82 ms
Wall time: 9.98 ms


In [4]:
%%time

df['title'] = df['title'].str.replace('\xa0', ' ')
df['title'] = df['title'].str.split('<').str.get(0)

CPU times: user 4.39 ms, sys: 4.18 ms, total: 8.57 ms
Wall time: 8.59 ms


In [5]:
%%time

split_years_of_life = df['years_of_life'].str.split('–')
df['year_of_birth'] = pd.to_numeric(split_years_of_life.str.get(0), errors='coerce')
df['year_of_death'] = pd.to_numeric(split_years_of_life.str.get(1), errors='coerce')
df['epoch'] = df[['year_of_birth', 'year_of_death']].mean(axis=1) // 100

CPU times: user 27.9 ms, sys: 5.3 ms, total: 33.2 ms
Wall time: 43.5 ms


In [6]:
%%time

df['text'] = ''
for i in df.index:
    with open(df.loc[i, 'filename'], 'r', encoding='utf-8') as f:
        df.loc[i, 'text'] = f.read().replace('\xad', ' ').replace('\xa0', ' ').replace('\n', ' ')

CPU times: user 2.76 s, sys: 690 ms, total: 3.45 s
Wall time: 6.21 s


In [7]:
STOPWORDS = nltk.corpus.stopwords.words('russian')

punctuation = set(punctuation + '«»—–…“”\n\t' + digits)
TABLE = str.maketrans({ch: ' ' for ch in punctuation})

In [8]:
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

In [9]:
df['text_tokenized'] = df.apply(lambda r: tokenize_text(r['text']), axis=1).values

In [10]:
mapping = {'COM': 'ADJ', 'APRO': 'DET', 'PART': 'PART', 'PR': 'ADP', 'ADV': 'ADV', 'INTJ': 'INTJ',
           'S': 'NOUN', 'V': 'VERB', 'CONJ': 'SCONJ', 'UNKN': 'X', 'ANUM': 'ADJ', 'NUM': 'NUM',
           'NONLEX': 'X', 'SPRO': 'PRON', 'ADVPRO': 'ADV', 'A': 'ADJ'}
pymystem = Mystem()

def pymystem_lemmatize_text(pymystem, text, mapping):
    lemmas = []
    lemmas_pos = []
    ana = pymystem.analyze(text.translate(TABLE))
    
    for word in ana:
        if word.get('analysis') and len(word.get('analysis')) > 0:
            lemma = word['analysis'][0]['lex'].lower().strip()
            
            if lemma not in STOPWORDS:
                lemmas.append(lemma)
                
                pos = word['analysis'][0]['gr'].split(',')[0]
                pos = pos.split('=')[0].strip()
                if pos in mapping:
                    lemmas_pos.append(lemma + '_' + mapping[pos]) # здесь мы конвертируем тэги
                else:
                    lemmas_pos.append(lemma + '_X') # на случай, если попадется тэг, которого нет в маппинге
    
    return lemmas, lemmas_pos

In [11]:
%%time

res = df.apply(lambda r: pymystem_lemmatize_text(pymystem, r['text'], mapping), axis=1).values
df_res = pd.DataFrame(list(res), columns = ['text_pymystem_list', 'text_pymystem_pos_list'])
df = df.join(df_res)

CPU times: user 23.8 s, sys: 381 ms, total: 24.2 s
Wall time: 2min 6s


In [12]:
%%time

df.to_pickle('metatable_preprocessed.pkl')

CPU times: user 2.16 s, sys: 516 ms, total: 2.68 s
Wall time: 3.72 s


In [13]:
def get_grouped_dataframe(df, col_to_groupby, first_arg, *args):
    grouped = df.groupby([col_to_groupby])
    
    def func(data):
        if isinstance(data.iloc[0], str):
            return ' '.join
        elif isinstance(data.iloc[0], list):
            return sum
    
    df_res = grouped[first_arg].agg(func(df[first_arg])).reset_index()
    for arg in args:
        df_res[arg] = grouped[arg].agg(func(df[arg])).reset_index()[arg]
        
    return df_res

In [14]:
%%time

# без колонки text (Memory Error)

traditions = get_grouped_dataframe(df, 'tradition',
                                   'text_tokenized',
                                   'text_pymystem_list',
                                   'text_pymystem_pos_list')
traditions.to_pickle('traditions.pkl')

CPU times: user 26.1 s, sys: 379 ms, total: 26.5 s
Wall time: 27.2 s


In [15]:
%%time

# без колонки text (Memory Error)

countries = get_grouped_dataframe(df, 'country',
                                  'text_tokenized', 
                                  'text_pymystem_list',
                                  'text_pymystem_pos_list')
countries.to_pickle('countries.pkl')

CPU times: user 8.37 s, sys: 385 ms, total: 8.75 s
Wall time: 9.44 s


In [16]:
%%time

traditions_topic_modeling = df.groupby(['tradition'])['text_pymystem_list'].apply(list).reset_index()
traditions_topic_modeling.to_pickle('traditions_topic_modeling.pkl')

CPU times: user 494 ms, sys: 84.5 ms, total: 578 ms
Wall time: 809 ms


In [17]:
%%time

grouped_by_authors = df.groupby(['author']).count()
needed_authors_list = grouped_by_authors.iloc[np.where(grouped_by_authors['text'] > 5)].index
needed_authors_grouped = df[df['author'].isin(needed_authors_list)].groupby(['author'])
needed_authors = needed_authors_grouped['text_pymystem_list'].apply(list).reset_index()

needed_authors.to_pickle('authors.pkl')

CPU times: user 231 ms, sys: 8.12 ms, total: 240 ms
Wall time: 258 ms


In [18]:
# test data for stylo

grouped = df.groupby(['name'])
needed_names_list = grouped.count().iloc[np.where(grouped['text_pymystem_list'].agg(sum).agg(len) >= 20000)].index
needed_names_grouped = df[df['name'].isin(needed_names_list)].groupby(['name'])
test = needed_names_grouped['text_pymystem_list'].apply(sum).reset_index()

In [19]:
needed_names_grouped['text_pymystem_list'].apply(sum).apply(len)

name
Д. А. Карельский    30229
М. Л. Гаспаров      28589
Name: text_pymystem_list, dtype: int64

In [20]:
for j in test.index:
    
    l = test.loc[j, 'text_pymystem_list']
    lst = [test.loc[j, 'text_pymystem_list'][i : i + 10000] for i in range(0, len(l) - (len(l) % 10000) , 10000)]

    for i, k in enumerate(lst):
        with open(os.getcwd() + '/corpus/_{}.txt'.format(test.loc[j, 'name'], i), 'w', encoding='utf-8') as f:
             f.write(' '.join(k))

In [21]:
df = pd.read_pickle('metatable_preprocessed.pkl')

In [22]:
grouped = df.groupby(['username'])
needed_names_list = grouped.count().iloc[np.where(grouped['text_pymystem_list'].agg(sum).agg(len) >= 20000)].index
needed_names_grouped = df[df['username'].isin(needed_names_list)].groupby(['username'])
test = needed_names_grouped['text_pymystem_list'].apply(sum).reset_index()

In [25]:
for j in test.index:
    
    l = test.loc[j, 'text_pymystem_list']
#     lst = [test.loc[j, 'text_pymystem_list'][i : i + 10000] for i in range(0, len(l) - (len(l) % 10000) , 10000)]

    with open(os.getcwd() + '/corpus/_{}.txt'.format(test.loc[j, 'username']), 'w', encoding='utf-8') as fw:
        fw.write(' '.join(l))

In [26]:
test['text_pymystem_list'] = test.apply(lambda r: ' '.join(r['text_pymystem_list']), axis=1).values
test.to_csv(os.getcwd() + '/corpus/data_stylo.tsv'.format(test.loc[j, 'username']), sep='\t', index=None)